In [1]:
import pulp
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/players_cleaned.csv')

In [3]:
player_names = df['name']
player_cost = df['cost']
player_points = df['points']
player_pos = df[['position_DEF', 'position_GKP', 'position_FWD', 'position_MID']]
teams = set(df.columns) - set(['name', 'cost', 'points', 'position_DEF', 'position_GKP', 'position_FWD', 'position_MID'])
player_team = df[list(teams)]

In [4]:
assert(len(player_team.columns) == 20)
assert(len(player_pos.columns) == 4)

In [5]:
player_binary_choice = [pulp.LpVariable(player_names[i], cat='Binary') for i in range(len(player_names))]

In [6]:
assert(len(player_binary_choice) == len(player_names))

In [7]:
FPL_problem = pulp.LpProblem("Maximize_Fantasy_Points", pulp.LpMaximize)

I will add the objective function to our problem

In [8]:
objective = 0
for i in range(len(player_binary_choice)):
    objective += player_binary_choice[i] * player_points[i]

FPL_problem += objective, 'Objective'

I will now add the position constraints

In [9]:
number_of = {'position_GKP': 2, 'position_FWD': 3, 'position_MID': 5, 'position_DEF': 5}
for pos in player_pos.columns:
    x = 0
    for i in range(len(player_binary_choice)):
        x += player_binary_choice[i] * player_pos[pos][i]
    FPL_problem += x == number_of[pos], 'Constraint_' + pos

I will now add the team constraint

In [10]:
for team in player_team.columns:
    x = 0
    for i in range(len(player_binary_choice)):
        x += player_binary_choice[i] * player_team[team][i]
    FPL_problem += x <= 3, 'Constraint_' + team

I will now add the cost constraint

In [11]:
x = 0
for i in range(len(player_binary_choice)):
    x += player_binary_choice[i] * player_cost[i]
FPL_problem += x <= 1000, 'Constraint_cost'

Solve the problem

In [12]:
FPL_problem.solve()

1

In [21]:
original_df = pd.read_csv('data/players.csv')
data = {'position': [], 'player': [], 'points': [], 'cost': [], 'team': []}

for i in range(len(player_binary_choice)):
    if pulp.value(player_binary_choice[i]) == 1:
        data['player'].append(player_names[i])
        data['cost'].append(player_cost[i])
        data['points'].append(player_points[i])
        data['team'].append(original_df['team'][i])
        data['position'].append(original_df['position'][i])

        
df = pd.DataFrame(data)
df['position'] = pd.Categorical(df['position'], categories=['GKP', 'DEF', 'MID', 'FWD'], ordered=True)
df.sort_values(by='position', inplace=True)
df

,position,player,points,cost,team
5,GKP,Mark Flekken,119,45,Brentford
9,GKP,Jordan Pickford,153,50,Everton
1,DEF,William Saliba,164,60,Arsenal
2,DEF,Benjamin White,182,65,Arsenal
7,DEF,Joachim Andersen,121,45,Crystal Palace
8,DEF,Jarrad Branthwaite,124,50,Everton
14,DEF,Pedro Porro,136,55,Spurs
0,MID,Declan Rice,165,65,Arsenal
6,MID,Cole Palmer,244,105,Chelsea
10,MID,Phil Foden,230,95,Man City
